<a href="https://colab.research.google.com/github/bkvkrll/-Introduction-to-Natural-Language-Processing/blob/main/lesson5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Тема «POS-tagger и NER»

Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы


In [25]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import pyconll

!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-08-19 08:01:02--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train-a.conllu.1’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   186MB/s    in 0.2s    

2022-08-19 08:01:02 (186 MB/s) - ‘ru_syntagrus-ud-train-a.conllu.1’ saved [40736565/40736565]

--2022-08-19 08:01:02--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [27]:
full_train = pyconll.load_from_file('/content/ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('/content/ru_syntagrus-ud-dev.conllu')

In [28]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    for word in sent:
        fdata_sent_test.append(word.form)

In [29]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [30]:
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [31]:
unigram_tagger = UnigramTagger(fdata_train)

In [32]:
unigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  """Entry point for launching an IPython kernel.


0.823732013802982

In [33]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.8292792499511688

In [34]:
trigram_tagger = TrigramTagger(fdata_train, backoff=unigram_tagger)
trigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.828550035809623

In [35]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.8291425222996289

In [36]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  if sys.path[0] == '':


0.827905462595221

In [37]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [38]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [39]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [40]:

le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [41]:
hvectorizer = HashingVectorizer(ngram_range=(1, 4), analyzer='char', n_features=100)

In [42]:
for number, element in enumerate(test_tok):
    if element == None:
        test_tok[number] = 'Договор'

In [43]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

X_train.shape

(426182, 100)

In [44]:
import xgboost
from xgboost import XGBClassifier

xgb_model = XGBClassifier()

In [45]:
xgb_model.fit(X_train, train_enc_labels)

XGBClassifier(objective='multi:softprob')

In [46]:
pred = xgb_model.predict(X_test)

In [47]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
test_enc_labels = le.fit_transform(test_label)

print(classification_report(test_enc_labels, pred))

              precision    recall  f1-score   support

           0       0.73      0.65      0.69     15103
           1       0.98      0.96      0.97     13717
           2       0.88      0.48      0.62      7783
           3       0.80      0.97      0.88      1390
           4       0.88      1.00      0.93      5672
           5       0.84      0.61      0.71      4265
           6       0.70      0.29      0.41        24
           7       0.66      0.91      0.76     36238
           8       1.00      0.77      0.87       265
           9       0.81      0.64      0.72      1734
          10       0.95      0.74      0.83      5125
          11       0.78      0.86      0.82      7444
          12       0.83      0.14      0.24      5473
          13       0.98      1.00      0.99     29186
          14       0.81      0.90      0.85      2865
          15       1.00      0.85      0.92        62
          16       0.81      0.63      0.71     17110
          17       0.36    

Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [48]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.4 MB/s 


In [49]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:

!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-08-19 08:16:11--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  2.01MB/s    in 0.9s    

2022-08-19 08:16:13 (2.01 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [51]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [52]:
from corus import load_ne5

path_coll5 = 'Collection5/'
records = load_ne5(path_coll5)
#next(records)

In [53]:
from razdel import tokenize

In [54]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [55]:
import pandas as pd

df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [56]:

df_words.shape

(265471, 2)

In [57]:
words_list = []
for word in words_docs:
    words_list.append(word[0])

words_list[:10]

['Главой',
 'КС',
 'Украины',
 'стал',
 'выходец',
 'из',
 'Донецка',
 'Председателем',
 'Конституционного',
 'суда']

In [58]:
import nltk
nltk.download('averaged_perceptron_tagger')

nltk_pos_tags = nltk.pos_tag(words_list)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [59]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

chunks = nltk.ne_chunk(nltk_pos_tags, binary=False)

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [60]:
len(words_list)

265471

In [61]:
pos_tag = nltk.pos_tag(words_list)
chunk = nltk.ne_chunk(pos_tag, binary=False)  
entities = []
labels = []
for chunk in chunks:
    if hasattr(chunk, 'label'):
        for c in chunk:
            entities.append(c[0])
            labels.append(chunk.label())
    else:
        entities.append(chunk[0])
        labels.append('OUT')

In [62]:
entities_labels = list(zip(entities, labels))
entities_df = pd.DataFrame(entities_labels, columns = ['Entities', 'Labels'])
entities_df

,Entities,Labels
0,Главой,PERSON
1,КС,ORGANIZATION
2,Украины,ORGANIZATION
3,стал,OUT
4,выходец,OUT
...,...,...
265466,должностей,OUT
265467,высшего,OUT
265468,начальствующего,OUT
265469,состава,OUT


In [63]:
df_words['nltk_ner'] = entities_df['Labels']
df_words

,word,tag,nltk_ner
0,Главой,OUT,PERSON
1,КС,ORG,ORGANIZATION
2,Украины,GEOPOLIT,ORGANIZATION
3,стал,OUT,OUT
4,выходец,OUT,OUT
...,...,...,...
265466,должностей,OUT,OUT
265467,высшего,OUT,OUT
265468,начальствующего,OUT,OUT
265469,состава,OUT,OUT


In [64]:
df_words['nltk_ner'].unique()

array(['PERSON', 'ORGANIZATION', 'OUT', 'GPE', 'GSP', 'FACILITY'],
      dtype=object)

In [65]:
df_words['tag'].unique()

array(['OUT', 'ORG', 'GEOPOLIT', 'LOC', 'PER', 'MEDIA'], dtype=object)

In [66]:
def ner_metrics(df, true_tag, ner_tag, column_name_1, column_name_2):
    right = 0
    fault = 0
    fault_2 = 0

    for index, person in  df.iterrows():
        if (person[column_name_1] == true_tag) & (person[column_name_2] == ner_tag):
            right += 1
        elif (person[column_name_1] == true_tag) & (person[column_name_2] !=  ner_tag):
            fault += 1
        elif (person[column_name_1] != true_tag) & (person[column_name_2] ==  ner_tag):
            fault_2 += 1
    precision = right / (right+fault_2)
    recall = right / (right+fault)
    f1 = (2*precision*recall) / (precision+recall)
    return print(f'precision = {precision}, recall = {recall}, f1_score = {f1}')

In [67]:
ner_metrics(df_words, 'PER', 'PERSON', 'tag', 'nltk_ner')

precision = 0.5959447358321607, recall = 0.4394811320754717, f1_score = 0.5058912960851386


In [68]:
ner_metrics(df_words, 'ORG', 'ORGANIZATION', 'tag', 'nltk_ner')

precision = 0.36153161175422976, recall = 0.05948282177129881, f1_score = 0.10215763980625275


In [69]:
!python -m spacy download ru_core_news_md

2022-08-19 08:28:11.844164: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.9 MB 1.2 MB/s 
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 9.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=3547213bd0792393e97f860a3e3a1ec5cedc4512af7e84cca4d01d131c21c331
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [70]:
import spacy

nlp = spacy.load('ru_core_news_md')

labels=[]
i=0
for word in words_list:
    spacy_ner = nlp(word)
    if spacy_ner.__getitem__(0).ent_type_ == '':
        labels.append('OUT')
    else:
        labels.append(spacy_ner.__getitem__(0).ent_type_)
    i+=1
    if (i % 10000) == 0:
        print(f'обработано {i} слов')

обработано 10000 слов
обработано 20000 слов
обработано 30000 слов
обработано 40000 слов
обработано 50000 слов
обработано 60000 слов
обработано 70000 слов
обработано 80000 слов
обработано 90000 слов
обработано 100000 слов
обработано 110000 слов
обработано 120000 слов
обработано 130000 слов
обработано 140000 слов
обработано 150000 слов
обработано 160000 слов
обработано 170000 слов
обработано 180000 слов
обработано 190000 слов
обработано 200000 слов
обработано 210000 слов
обработано 220000 слов
обработано 230000 слов
обработано 240000 слов
обработано 250000 слов
обработано 260000 слов


In [71]:
len(labels)

265471

In [72]:

df_words['spacy_ner'] = labels

In [73]:
df_words

,word,tag,nltk_ner,spacy_ner
0,Главой,OUT,PERSON,OUT
1,КС,ORG,ORGANIZATION,ORG
2,Украины,GEOPOLIT,ORGANIZATION,LOC
3,стал,OUT,OUT,OUT
4,выходец,OUT,OUT,OUT
...,...,...,...,...
265466,должностей,OUT,OUT,OUT
265467,высшего,OUT,OUT,OUT
265468,начальствующего,OUT,OUT,OUT
265469,состава,OUT,OUT,OUT


In [74]:
ner_metrics(df_words, 'PER', 'PER', 'tag', 'spacy_ner')

precision = 0.9747936849659131, recall = 0.5125943396226416, f1_score = 0.6718807963398046


In [75]:
ner_metrics(df_words, 'ORG', 'ORG', 'tag', 'spacy_ner')

precision = 0.6543102195086495, recall = 0.3297194344736649, f1_score = 0.4384802727715538


In [76]:
ner_metrics(df_words, 'LOC', 'LOC', 'tag', 'spacy_ner')

precision = 0.22163773452568825, recall = 0.276707530647986, f1_score = 0.246129880245351
